In [90]:
from __future__ import print_function
import keras
import numpy as np
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt
import json

In [105]:
batch_size = 10
epochs = 10
channels = 5
grid_width, grid_height = 10, 10

In [106]:
input_data = json.load(open('../data/citywars.log'))

In [112]:
multi_channel_input_data = np.asarray([np.stack([
    np.asarray(sample['cities']).reshape(grid_width, grid_height),
    np.asarray(sample['player0']).reshape(grid_width, grid_height),
    np.asarray(sample['player1']).reshape(grid_width, grid_height),
    np.asarray(sample['obstacles']).reshape(grid_width, grid_height),
    np.asarray(sample['trajectory']).reshape(grid_width, grid_height)
], axis=-1) for sample in input_data])

labels = [sample['arrival'] for sample in input_data]

In [111]:
x_train = np.asarray([multi_channel_input_data[0] for x in range(1000)])
x_test = np.asarray([multi_channel_input_data[0] for x in range(100)])

y_train = np.asarray([labels[0] for x in range(1000)])
y_test = np.asarray([labels[0] for x in range(100)])

In [109]:
input_shape = (grid_width, grid_height, channels)

In [ ]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=keras.losses.mean_squared_error,
              optimizer=keras.optimizers.Adam(),
              metrics=['mae'])


class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.mae = []

    def on_epoch_end(self, batch, logs={}):
        self.mae.append(logs.get('mean_absolute_error'))


history = AccuracyHistory()

model.fit(x_train, y_train,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test mae:', score[1])
plt.plot(range(1, epochs+1), history.mae)
plt.xlabel('Epochs')
plt.ylabel('Mae')
plt.show()

x_train shape: (1000, 10, 10, 5)
1000 train samples
100 test samples


In [121]:
history.mae

[None, None, None, None, None, None, None, None, None, None]